# Spotify Playlist Generator

In [1]:
import requests
import base64
import hashlib
import os
import json
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [2]:
# Spotify API credentials
client_id = 'your_client_id_here'
redirect_uri = 'your_redirect_uri_here'

# Generate a random string as the code verifier for PKCE
code_verifier = base64.urlsafe_b64encode(os.urandom(32)).decode().rstrip('=')
code_challenge = base64.urlsafe_b64encode(hashlib.sha256(code_verifier.encode()).digest()).decode().rstrip('=')

# Spotify authorization endpoint
authorization_url = 'https://accounts.spotify.com/authorize'
auth_params = {
    'client_id': client_id,
    'response_type': 'code',
    'redirect_uri': redirect_uri,
    'code_challenge_method': 'S256',
    'code_challenge': code_challenge,
    'scope': 'user-library-read user-follow-read playlist-modify-private playlist-modify-public playlist-read-private user-read-private user-read-email'
}#Add the scope based on the information you choose to share
authorization_response = requests.get(authorization_url, params=auth_params)

# Redirect the user to Spotify for authorization
print("Please authorize the application by visiting the following URL:")
print(authorization_response.url)

Please authorize the application by visiting the following URL:
https://accounts.spotify.com/en/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%3Fscope%3Duser-library-read%2Buser-follow-read%2Bplaylist-modify-private%2Bplaylist-modify-public%2Bplaylist-read-private%2Buser-read-private%2Buser-read-email%26response_type%3Dcode%26redirect_uri%3Dhttps%253A%252F%252Fshegszz.github.io%252FSpotify-Playlist-Generator%252F%26code_challenge_method%3DS256%26client_id%3Da5203ec276044c97a6ec5389248f8b79%26code_challenge%3Dw6IwBVBKBlMcbrPUNXmw0hhJfcYnzWKxd_sbj7Y1fhk


In [3]:
# After authorization, Spotify will redirect back to your redirect URI with the authorization code
authorization_code = input("Enter the authorization code from the redirected URL: ")

# Exchange authorization code for access token using PKCE
token_url = 'https://accounts.spotify.com/api/token'
token_params = {
    'grant_type': 'authorization_code',
    'code': authorization_code,
    'redirect_uri': redirect_uri,
    'client_id': client_id,
    'code_verifier': code_verifier
}
token_response = requests.post(token_url, data=token_params)

if token_response.status_code == 200:
    access_token = token_response.json()['access_token']
    refresh_token = token_response.json().get('refresh_token')


Enter the authorization code from the redirected URL: 


In [4]:
# Get all liked songs(handle pagination)
offset = 0
limit = 50  # Maximum limit per request is 50

# Initialize an empty list to store all liked songs
all_liked_songs = []

# Loop until all liked songs are fetched
while True:
    liked_songs_response = requests.get(
        'https://api.spotify.com/v1/me/tracks',
        headers={'Authorization': 'Bearer ' + access_token},
        params={'limit': limit, 'offset': offset}
    )
    liked_songs_data = liked_songs_response.json()
    if 'items' not in liked_songs_data or len(liked_songs_data['items']) == 0:
        # No more songs left to fetch
        break
    all_liked_songs.extend(liked_songs_data['items'])
    offset += limit
print(all_liked_songs)

In [5]:
import pandas as pd
# Extract relevant data from all liked songs
song_data = []

for song in all_liked_songs:
    Track_name = song['track']['name']  # Extract track name
    Artist_names = [artist['name'] for artist in song['track']['artists']]  # Extract artist names
    Album_name = song['track']['album']['name']  # Extract album name
    Popularity = song['track']['popularity'] # Extract popularity score
    Track_id = song['track']['id'] # Extract track id
    Date_Added = song['added_at'] # Extract date added
    
#check if features key exists
    features = song['track'].get('features', None)
    if features:
        danceability = features['danceability']
        energy = features['energy']
        valence = features['valence']
        tempo = features['tempo']
    else: # set to None if features not available
        danceability = None
        energy = None
        valence = None
        tempo = None
       
    # Still access genres 
    genres = ', '.join(song['track'].get('genres', []))

        
    song_data.append({
        'Track': Track_name,
        'Artist': Artist_names,
        'Album': Album_name,
        'Popularity': Popularity,
        'Danceability': danceability,
        'Energy': energy,
        'Valence': valence,
        'Tempo': tempo,
        'id': Track_id,
        'Date Added': Date_Added,
        'Genres': genres if genres else 'N/A'
    })

#Create DataFrame from song data
all_liked_songs_df = pd.DataFrame(song_data)
print(song_data)

In [6]:
# Sort the songs based on popularity
sorted_songs = sorted(song_data, key=lambda x: x['Popularity'], reverse=True)

# Create a new playlist
playlist_name = 'Repeat on Shuffle: My Top 100'
create_playlist_url = 'https://api.spotify.com/v1/me/playlists'
create_playlist_headers = {
                'Authorization': f'Bearer {access_token}',
                'Content-Type': 'application/json'
}
create_playlist_data = {
                'name': 'Repeat on Shuffle: My Top 100',
                'public': True  # Change to True if you want the playlist to be public
            }

#Convert create_playlist_data to Json format
create_playlist_data_json = json.dumps(create_playlist_data)

#send request to create the playlist
create_playlist_response = requests.post(create_playlist_url, headers=create_playlist_headers, data=create_playlist_data_json)

#check if the playlist was created successfully
if create_playlist_response.status_code == 201:
    print(f"Playlist '{playlist_name}' created successfully!")



Playlist 'Repeat on Shuffle: My Top 100' created successfully!


In [7]:
#Extract the playlist ID from the response
playlist_id = create_playlist_response.json()['id']
    
# Add the top 100 songs by popularity to the new playlist
tracks_to_add = [song['id'] for song in sorted_songs[:100]]

add_tracks_url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
add_tracks_data = {
        'uris': [f'spotify:track:{track_id}' for track_id in tracks_to_add]
}

add_tracks_response = requests.post(add_tracks_url, headers=create_playlist_headers, json=add_tracks_data)
if add_tracks_response.status_code == 201:
        print(f"Top 100 songs added to the playlist successfully!")
    

In [10]:
#Create DataFrame from playlist created
My_Playlist_Project = sorted_songs[:100]
df = pd.DataFrame(My_Playlist_Project)
df
# Spotify lacks a direct genre API endpoint

,Track,Artist,Album,Popularity,Danceability,Energy,Valence,Tempo,id,Date Added,Genres
0,Like That,"[Future, Metro Boomin, Kendrick Lamar]",WE DON'T TRUST YOU,96,None,None,None,None,2tudvzsrR56uom6smgOcSf,2024-04-01T06:47:28Z,N/A
1,Beautiful Things,[Benson Boone],Beautiful Things,96,None,None,None,None,6tNQ70jh4OwmPGpYy6R2o9,2024-03-22T22:44:09Z,N/A
2,euphoria,[Kendrick Lamar],euphoria,94,None,None,None,None,77DRzu7ERs0TX3roZcre7Q,2024-05-01T07:22:08Z,N/A
3,FE!N (feat. Playboi Carti),"[Travis Scott, Playboi Carti]",UTOPIA,93,None,None,None,None,42VsgItocQwOQC3XWZ8JNA,2023-12-31T10:03:50Z,N/A
4,Push Ups,[Drake],Push Ups,91,None,None,None,None,3eh51r6rFWAlGQRlHx9QnQ,2024-04-19T17:22:58Z,N/A
...,...,...,...,...,...,...,...,...,...,...,...
95,Hours In Silence,"[Drake, 21 Savage]",Her Loss,73,None,None,None,None,0sSRLXxknVTQDStgU1NqpY,2023-05-29T06:25:17Z,N/A
96,Way 2 Sexy (with Future & Young Thug),"[Drake, Future, Young Thug]",Certified Lover Boy,73,None,None,None,None,0k1WUmIRnG3xU6fvvDVfRG,2023-02-06T21:45:38Z,N/A
97,They Don't Care About Us,[Michael Jackson],"HIStory - PAST, PRESENT AND FUTURE - BOOK I",73,None,None,None,None,3wuCCNCnBhJlwkIJTBZFiv,2023-01-02T19:13:43Z,N/A
98,"I'm the One (feat. Justin Bieber, Quavo, Chanc...","[DJ Khaled, Justin Bieber, Quavo, Chance the R...",Grateful,72,None,None,None,None,3DXncPQOG4VBw3QHh3S817,2024-02-27T09:29:43Z,N/A


In [11]:
# Saving DataFrame to csv
df.to_csv('My Top100 spotify playlist data.csv', index=False)

In [12]:
# Get user playlists
playlists_url = 'https://api.spotify.com/v1/me/playlists'
playlists_headers = {
        'Authorization': f'Bearer {access_token}'
}
playlists_response = requests.get(playlists_url, headers=playlists_headers)

if playlists_response.status_code == 200:
    playlists_data = playlists_response.json()
    print("User Playlists:")
    for playlist in playlists_data['items']:
        print(playlist['name'])


User Playlists:
Ughhh
Repeat on Shuffle: My Top 100
Repeat Rewind
On Repeat
My recommendation playlist
Horladee + D_grey
Beast Mode Hip-Hop
Most Streamed Rap Songs on Spotify
Dwizztheman
Your Top Songs 2023
Motivation
Comedy
Jazz_Soul_Blues
Rap Workout
Blues Classics
Favorites
Billboard Hot 100
The GRAMMYs Official Playlist
I Love My '00s R&B
Gospel
RNB WORKOUT
Power Soundtrack
J. Cole Mix
I Love My '90s Hip-Hop
Top 50 - Global
